In [1]:
# Test MPC

# Standard imports for SCP via JAX and CVXPY
import numpy as np # CVXPY uses numpy
import cvxpy as cvx # For Convex Optimization
from cvxpy.constraints import constraint
import jax # For computing gradients and JIT
import jax.numpy as jnp
from tqdm import tqdm # For progress bars
from functools import partial # For helping JAX

# Assessment
import time
import matplotlib.pyplot as plt
import scipy.interpolate
import scipy.optimize

import dynamics as dn
import control as ct

## Common Parameters

In [2]:
import pykep
# SPACECRAFT PARAMETERS
C_D = 1.5 # Dimensionless
C_SRP = 1.5 # Dimensionless

A = 1.0   # m^2
MASS_SAT = 100   # kg

GAMMA_SRP = C_SRP * A / MASS_SAT
GAMMA_DRAG = C_D * A / MASS_SAT

PSI = 0    # thermal oscillation

R_EARTH = 6378
MU_EARTH_KM = pykep.MU_EARTH * 1e-9
EARTH_TO_SUN_VEC = dn.compute_earth_to_sun()


## Spacecraft Initial States

In [3]:
# Initial condition of reference orbit
h = 300  # km  height for perigee
e = 0.001
a = (R_EARTH + h) / (1 - e)
i = 0.6
W = 0
w = 0
E = 0

# satellite initial condition
dr = np.array([1.2e-5, 1.5e-5, 1.3e-5])   # relative position vector to ref
dv = np.array([1.1e-5, 1.4e-5, 1.7e-5])  # relative velocity to ref

# orbital period
T = 2 * jnp.pi * jnp.sqrt(a**3/MU_EARTH_KM)

# compute orbit elements
oe_chief = np.array([a, e, i, W, w, E])
r0_chief, v0_chief = pykep.core.par2ic(oe_chief, MU_EARTH_KM)
r0_sat, v0_sat = r0_chief + dr, v0_chief + dv

# convert to JAX array
x0_chief_eci = jnp.array(np.hstack([r0_chief, v0_chief]))
x0_sat_eci = jnp.array(np.hstack([r0_sat, v0_sat]))

# Initial  States
s0_rel_sat = jnp.hstack([dn.eci_to_lvlh(x0_chief_eci, x0_sat_eci)])
print(f"Should be small but not zero = {s0_rel_sat}")
s0_rel_chief = jnp.hstack([dn.eci_to_lvlh(x0_chief_eci, x0_chief_eci)])
print(f"Should be uniformly zero = {s0_rel_chief}")

Should be small but not zero = [0.0000000e+00 1.9720386e-05 2.2597262e-06 1.1022827e-05 2.1499252e-05
 6.0905495e-06]
Should be uniformly zero = [0. 0. 0. 0. 0. 0.]


## Time Propagation Settings

In [4]:
# Propagation time settings
dt = 0.01
tspan_quarter_periods = np.arange(0, 0.025 * T, dt)   # propagation step
print(f"Tspan size: {tspan_quarter_periods.size}")

Tspan size: 13598


## JIT Dynamics Functions

In [5]:
# JIT the dynamics functions
t = 0  # we can assume this because there is no change in the parameters
fd_absolute = jax.jit(ct.discretize(ct.absolute_dynamics, dt))

x_chief_eci_traj = ct.propagate_absolute_from_control(x0_chief_eci, ct.zero_control, tspan_quarter_periods, fd_absolute)

interp_abs_chief_jit = jax.jit(ct.interp_fixed(tspan_quarter_periods, x_chief_eci_traj[:, :-1]))
fd_relative_4T = jax.jit(ct.discretize(ct.relative_dynamics, dt, params=(GAMMA_SRP, GAMMA_DRAG, PSI), state_eci_ref_func=interp_abs_chief_jit))

## Control Constraints

In [6]:
# Test optimizing with scipy minimize
min_T = -250e-2  # N
max_T = 250e-2  # N

u_min = min_T/MASS_SAT  # m/s^2
u_max = max_T/MASS_SAT  #m/s^2

print("umax [m/s]:", u_max)

umax [m/s]: 0.025


## Try Sequential Convex Programming

In [10]:
# SCP parameters
P_outer = 1e10 * np.eye(6)                    # terminal state cost matrix
Q_outer = 1e7 * np.eye(6)                    # state cost matrix
rho_outer = 1.                               # trust region parameter
u_max_outer = u_max                          # control effort bound
tol = 5e-1                           # convergence tolerance
max_iters = 10                       # maximum number of SCP iterations
u_dim_outer = 3

# Solve the satellite tracking problem with SCP
N_horizon = 40
t_scp = np.arange(0, (N_horizon + 1) * dt, dt)
N_horizon = t_scp.size - 1
print(f"Horizon is {N_horizon} steps")
s_scp, u_scp = ct.solve_satellite_scp(fd_relative_4T, t_scp, P_outer, Q_outer, N_horizon, s0_rel_sat,
                           u_max_outer, rho_outer, tol, max_iters, u_dim_outer)

print(f"\n\n Shapes: s={s_scp.shape} and {u_scp.shape}")

Horizon is 40 steps
ITERATION FAILED!! message is:

 Solver 'OSQP' failed. Try another solver, or solve with verbose=True for more information.
                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) May 31 12:15:23 PM: Your problem has 366 variables, 43 constraints, and 0 parameters.
(CVXPY) May 31 12:15:23 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 31 12:15:23 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 31 12:15:23 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 31 12:

/Users/keidaiiiyama/opt/anaconda3/envs/geccoenv/lib/python3.8/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Optimal Inaccurate iteration
iteration1 objective change: inf
ITERATION FAILED!! message is:

 Solver 'OSQP' failed. Try another solver, or solve with verbose=True for more information.
                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) May 31 12:16:53 PM: Your problem has 366 variables, 83 constraints, and 0 parameters.
(CVXPY) May 31 12:16:53 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 31 12:16:53 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 31 12:16:53 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
--------------------------------------------------------

## Finally, Run MPC

In [ ]:
N_total_mpc = 50
t_last_mpc = N_total_mpc * dt
N_horizon_mpc = 10
t_mpc, s_mpc, u_mpc = ct.run_MPC(s0_rel_sat, t_last_mpc, N_horizon_mpc,
                              P_outer, Q_outer, u_max, rho_outer,
                              tol, max_iters, u_dim_outer, dt, fd_relative_4T)

## Plot results

In [ ]:
ct.plot_mpc_control(t_mpc, s_mpc, u_mpc, u_max)

In [ ]:
ct.plot_mpc_trajectory(s_mpc, N_total_mpc, N_horizon_mpc)